In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import requests
import dateutil.parser as dp

# Introduction

### What is this?
This notebook uses the FTX API to find basis trades that can exist on the platform when there's a difference between the price of spot and a future. 

### Why would I want to do this?
In countries where trading cryptocurrency derivatives is possible, you can find arbitrage opportunities when the price of spot differs from that of a future. By longing spot and shorting a dated future, you can profit the difference at the time of the future expiry by selling your spot to the TWAP at the same time the futures contract is resolving. At time of writing, it is possible for these differences to be as much as +4.6% for contracts expiring 6 months in the future. Perpetual futures are different because they don't expire, but this notebook will provide the values for them too. 

## Settings

In [49]:
syms = ['BTC', 'ETH', 'SOL', 'TOMO'] # The tickers you want to examine

FTX_API_URI = 'https://ftx.com/api' # You may want to use FTX.us if that's more relevant to you and they've released derivatives

## Functions

In [50]:
def get_all_futures():
    url = FTX_API_URI + '/futures'
    r = requests.get(url).json()['result']
    return r

def get_spot_price(sym):
    url = FTX_API_URI + '/markets/' + sym + '/USD'
    r = requests.get(url).json()['result']
    return r['last']

def get_future_premiums(syms):
    futures = get_all_futures()
    for sym in syms:
        spot_price = get_spot_price(sym)
        print(sym + ' (Current price: ' + str(spot_price) + '): ')
        for future in futures:
            if future['underlying'] == sym:
                if 'Future' in future['description']:
                    premium = (future['last'] - spot_price) / future['last'] * 100
                    print(future['description'] + ': ' + "{:.2f}".format(round(premium, 2)) + '%')
        print('\n')

        

## The 'Go' button

In [51]:
get_future_premiums(syms)

BTC (Current price: 47367.0): 
Bitcoin Perpetual Futures: 0.01%
Bitcoin March 2022 Futures: 1.95%
Bitcoin June 2022 Futures: 4.57%


ETH (Current price: 3757.9): 
Ethereum Perpetual Futures: 0.01%
Ethereum March 2022 Futures: 2.07%
Ethereum June 2022 Futures: 4.62%


SOL (Current price: 176.23): 
Solana Perpetual Futures: 0.00%
Solana March 2022 Futures: 1.86%


TOMO (Current price: 1.9945): 
TomoChain Perpetual Futures: -0.03%


